In [1]:
import os
import subprocess
import glob
import mdtraj as md
import numpy as np
from natsort import natsorted
import re

In [2]:
pdb_dir = "data/PDBs"
gro_dir = "data/GROs"
cache_dir = "./Cache"
os.makedirs(cache_dir, exist_ok=True)
os.makedirs(gro_dir, exist_ok=True)

In [3]:
########################################### move to interface
z_adjust = np.load('z_adjust.npy')

gol_file_path =  "Gol.gro"
Au_PBC = [4.10200, 4.05993, 4.95517]  # Gold periodic boundary conditions
Au_Z = 4.0  # Gold surface at 4 nm
dis2Au = 0.5  # Buffer zone due to soft-core repulsion
target_Z = Au_Z - dis2Au  # Target COM height

gro_files = [f for f in os.listdir(gro_dir) if f.endswith("_processed.gro")]
with open(gol_file_path, "r") as f:
    gol_lines = f.readlines()
header = gol_lines[0]  # Title line
box_size = gol_lines[-1]  # Box dimensions
num_atoms_gol = int(gol_lines[1].strip())  # Extract atom count from Gol.gro
gol_molecules = gol_lines[2:-1]  # Exclude header and box size line

for gro_file in natsorted(gro_files):
    raw_file_path = os.path.join(gro_dir, gro_file)
    traj = md.load(raw_file_path)
    ###
    masses = np.array([atom.element.mass for atom in traj.topology.atoms]) # Compute center of mass (COM)
    com = np.sum(traj.xyz[0] * masses[:, np.newaxis], axis=0) / np.sum(masses)
    com_Z = com[2]
    index = int(gro_file.split('_')[-2])-1
    z_adjust_frame = z_adjust[index]
    trans_Z = target_Z - com_Z + z_adjust_frame
    ###
    peptide_xyz = traj.xyz     # Apply translation
    peptide_xyz[:, :, 2] += trans_Z
    peptide_xyz[:, :, 0] += Au_PBC[0] / 2
    peptide_xyz[:, :, 1] += Au_PBC[1] / 2
    ###
    transformed_gro_path = os.path.join(cache_dir, f"{gro_file.replace('_processed.gro', '_trans.gro')}") # Save translated structure
    traj.save_gro(transformed_gro_path)
    with open(transformed_gro_path, "r") as f:  # Merge with Gol.gro
        klv_lines = f.readlines()
    klv_molecules = klv_lines[2:-1]  # Exclude header and box size line
    total_atoms = num_atoms_gol + len(klv_molecules)  # Update atom count
    merged_file_path = os.path.join(gro_dir, gro_file.replace("_processed.gro", "_Gol_merged.gro"))
    ###
    with open(merged_file_path, "w") as f:
        f.write(header)  # Keep original title
        f.write(f" {total_atoms}\n")  # Update total atom count
        f.writelines(gol_molecules)  # Write Gol molecules
        f.writelines(klv_molecules)  # Append KLV molecules
        f.write(box_size)  # Write box dimensions

########################################### remove teriminal Hs
gro_files = [f for f in os.listdir(gro_dir) if f.endswith("_Gol_merged.gro")]

# Atom indices to remove (modify as needed)
indices_to_remove = np.array([3261, 3138, 3139]) - 1  # Convert to 0-based indexing

def run_gmx_editconf(input_gro, output_gro):
    command = ['gmx', 'editconf', '-f', input_gro, '-o', output_gro]
    subprocess.run(command, check=True)

for gro_file in gro_files:
    gro_file_path = os.path.join(gro_dir, gro_file)
    traj = md.load(gro_file_path)
    ###
    indices_to_remove_valid = indices_to_remove[indices_to_remove < traj.n_atoms]
    mask = np.ones(traj.n_atoms, dtype=bool)
    mask[indices_to_remove_valid] = False
    ###
    xyz = traj.xyz[:, mask, :]
    topology = traj.topology.subset(np.where(mask)[0])
    traj_truncated = md.Trajectory(xyz, topology)
    ###
    truncated_gro_path = os.path.join(cache_dir, gro_file.replace(".gro", "_truncated.gro"))
    traj_truncated.save_gro(truncated_gro_path)
    ###
    final_gro_path = os.path.join(cache_dir, gro_file.replace(".gro", "_reset.gro"))
    run_gmx_editconf(truncated_gro_path, final_gro_path)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3258 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.


              :-) GROMACS - gmx editconf, 2022.5-plumed_2.9.0 (-:

Executable:   /home/yanbin/Apps/gromacs-2022.5/build/bin/gmx
Data prefix:  /home/yanbin/Apps/gromacs-2022.5 (source tree)
Working dir:  /home/yanbin/Desktop/Projects/GolP/Full_data_code/4_18/Au_F_2/2_Z
Command line:
  gmx editconf -f ./Cache/KLVFFAE_2_AA_38_Gol_merged_truncated.gro -o ./Cache/KLVFFAE_2_AA_38_Gol_merged_reset.gro


Back Off! I just backed up ./Cache/KLVFFAE_2_AA_38_Gol_merged_reset.gro to ./Cache/#KLVFFAE_2_AA_38_Gol_merged_reset.gro.2#

GROMACS reminds you: "I'm a strong believer that ignorance is important in science. If you know too much, you start seeing reasons why things won't work. That's why its important to change your field to collect more ignorance." (Sydney Brenner)

              :-) GROMACS - gmx editconf, 2022.5-plumed_2.9.0 (-:

Executable:   /home/yanbin/Apps/gromacs-2022.5/build/bin/gmx
Data prefix:  /home/yanbin/Apps/gromacs-2022.5 (source tree)
Working dir:  /home/yanbin/Desktop/Proje

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3258 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3258 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.



GROMACS reminds you: "I'm a strong believer that ignorance is important in science. If you know too much, you start seeing reasons why things won't work. That's why its important to change your field to collect more ignorance." (Sydney Brenner)

              :-) GROMACS - gmx editconf, 2022.5-plumed_2.9.0 (-:

Executable:   /home/yanbin/Apps/gromacs-2022.5/build/bin/gmx
Data prefix:  /home/yanbin/Apps/gromacs-2022.5 (source tree)
Working dir:  /home/yanbin/Desktop/Projects/GolP/Full_data_code/4_18/Au_F_2/2_Z
Command line:
  gmx editconf -f ./Cache/KLVFFAE_2_AA_72_Gol_merged_truncated.gro -o ./Cache/KLVFFAE_2_AA_72_Gol_merged_reset.gro


Back Off! I just backed up ./Cache/KLVFFAE_2_AA_72_Gol_merged_reset.gro to ./Cache/#KLVFFAE_2_AA_72_Gol_merged_reset.gro.2#

GROMACS reminds you: "I'm a strong believer that ignorance is important in science. If you know too much, you start seeing reasons why things won't work. That's why its important to change your field to collect more ignorance." 

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3258 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3258 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.


...

              :-) GROMACS - gmx editconf, 2022.5-plumed_2.9.0 (-:

Executable:   /home/yanbin/Apps/gromacs-2022.5/build/bin/gmx
Data prefix:  /home/yanbin/Apps/gromacs-2022.5 (source tree)
Working dir:  /home/yanbin/Desktop/Projects/GolP/Full_data_code/4_18/Au_F_2/2_Z
Command line:
  gmx editconf -f ./Cache/KLVFFAE_2_AA_67_Gol_merged_truncated.gro -o ./Cache/KLVFFAE_2_AA_67_Gol_merged_reset.gro


Back Off! I just backed up ./Cache/KLVFFAE_2_AA_67_Gol_merged_reset.gro to ./Cache/#KLVFFAE_2_AA_67_Gol_merged_reset.gro.2#

GROMACS reminds you: "Love is Like Moby Dick, Get Chewed and Get Spat Out" (Urban Dance Squad)



Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3258 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3258 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.


              :-) GROMACS - gmx editconf, 2022.5-plumed_2.9.0 (-:

Executable:   /home/yanbin/Apps/gromacs-2022.5/build/bin/gmx
Data prefix:  /home/yanbin/Apps/gromacs-2022.5 (source tree)
Working dir:  /home/yanbin/Desktop/Projects/GolP/Full_data_code/4_18/Au_F_2/2_Z
Command line:
  gmx editconf -f ./Cache/KLVFFAE_2_AA_24_Gol_merged_truncated.gro -o ./Cache/KLVFFAE_2_AA_24_Gol_merged_reset.gro


Back Off! I just backed up ./Cache/KLVFFAE_2_AA_24_Gol_merged_reset.gro to ./Cache/#KLVFFAE_2_AA_24_Gol_merged_reset.gro.2#

GROMACS reminds you: "Love is Like Moby Dick, Get Chewed and Get Spat Out" (Urban Dance Squad)

              :-) GROMACS - gmx editconf, 2022.5-plumed_2.9.0 (-:

Executable:   /home/yanbin/Apps/gromacs-2022.5/build/bin/gmx
Data prefix:  /home/yanbin/Apps/gromacs-2022.5 (source tree)
Working dir:  /home/yanbin/Desktop/Projects/GolP/Full_data_code/4_18/Au_F_2/2_Z
Command line:
  gmx editconf -f ./Cache/KLVFFAE_2_AA_18_Gol_merged_truncated.gro -o ./Cache/KLVFFAE_2_AA_18_Go

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3258 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3258 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3258 atoms
Volume: 0 nm^3, corresponds to roughly 0 electrons
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.



GROMACS reminds you: "In science it often happens that scientists say, 'You know that's a really good argument; my position is mistaken,' and then they would actually change their minds and you never hear that old view from them again. They really do it. It doesn't happen as often as it should, because scientists are human and change is sometimes painful. But it happens every day. I cannot recall the last time something like that happened in politics or religion." (Carl Sagan)

              :-) GROMACS - gmx editconf, 2022.5-plumed_2.9.0 (-:

Executable:   /home/yanbin/Apps/gromacs-2022.5/build/bin/gmx
Data prefix:  /home/yanbin/Apps/gromacs-2022.5 (source tree)
Working dir:  /home/yanbin/Desktop/Projects/GolP/Full_data_code/4_18/Au_F_2/2_Z
Command line:
  gmx editconf -f ./Cache/KLVFFAE_2_AA_79_Gol_merged_truncated.gro -o ./Cache/KLVFFAE_2_AA_79_Gol_merged_reset.gro


Back Off! I just backed up ./Cache/KLVFFAE_2_AA_79_Gol_merged_reset.gro to ./Cache/#KLVFFAE_2_AA_79_Gol_merged_reset

In [4]:
######################################## MD
res_names = ['GLU','LEU','LYS','PHE','VAL']
# Define indices for each residue manually (adjust as needed)
# here SC include all atoms in the residual 
# default values as from MD of dipepetide
# the indecis already from 0
MD_res_indices = {
    'GLU': {
        'Ca_indices': np.array([0, 8, 23]),  # Example indices for GLU
        'SC_indices': np.arange(6, 21)      # Example indices for GLU side chain
    },
    'LEU': {
        'Ca_indices': np.array([0, 8, 27]), # Example indices for LEU
        'SC_indices': np.arange(6, 25)      # Example indices for LEU side chain
    },
    'LYS': {
        'Ca_indices': np.array([0, 8, 30]), # Example indices for LYS
        'SC_indices': np.arange(6, 28)      # Example indices for LYS side chain
    },
    'PHE': {
        'Ca_indices': np.array([0, 8, 28]), # Example indices for PHE
        'SC_indices': np.arange(6, 26)      # Example indices for PHE side chain
    },
    'VAL': {
        'Ca_indices': np.array([0, 8, 24]), # Example indices for VAL
        'SC_indices': np.arange(6, 22)      # Example indices for VAL side chain
    }
}
######################################## Str2Str
# sequence LYS-LEU-VAL-PHE-PHE-ALA-GLU
# Define original Ca indices, start from 1, PDB
# -1 means ends
# here SC include all atoms in the residual 
peptide_sc = [1,2,3,4,5,7]
Ca_indices_original = {
    1: {'res_name': 'LYS', 'Ca_indices': np.array([-1, 3139, 3161]), 'SC_indices': np.arange(3137, 3159)},
    2: {'res_name': 'LEU', 'Ca_indices': np.array([3139, 3161, 3180]), 'SC_indices': np.arange(3159, 3178)},
    3: {'res_name': 'VAL', 'Ca_indices': np.array([3161, 3180, 3196]), 'SC_indices': np.arange(3178, 3194)},
    4: {'res_name': 'PHE', 'Ca_indices': np.array([3180, 3196, 3216]), 'SC_indices': np.arange(3194, 3214)},
    5: {'res_name': 'PHE', 'Ca_indices': np.array([3196, 3216, 3236]), 'SC_indices': np.arange(3214, 3234)},
    7: {'res_name': 'GLU', 'Ca_indices': np.array([3236, 3246, -1]), 'SC_indices': np.arange(3244, 3259)},
}
S2S_residue_indices = {
    res_num: {'res_name': info['res_name'], 'Ca_indices': info['Ca_indices'] - 1, 'SC_indices': info['SC_indices'] - 1}
    for res_num, info in Ca_indices_original.items()
}

In [5]:
# get the xyz intact early is neat but slow
def load_MD(res_name):
    gro_file = f"./Traj/package_{res_name}/protein.gro"
    xtc_file = f"./Traj/package_{res_name}/traj_comp.xtc"
    # Load trajectory
    traj = md.load(xtc_file, top=gro_file)
    PBC = traj.unitcell_lengths  # Shape: (n_frames, 3) → (Lx, Ly, Lz) per frame
    # Extract relevant atoms
    Ca_indices = MD_res_indices[res_name]['Ca_indices']
    SC_indices = MD_res_indices[res_name]['SC_indices']
    xyz_Ca = traj.atom_slice(Ca_indices).xyz  # Shape: (n_frames, len(Ca_indices), 3)
    xyz_SC = traj.atom_slice(SC_indices).xyz  # Shape: (n_frames, len(SC_indices), 3)
    ###### **Screening Ca based on z-coordinate** ######
    distance_xyz_Ca = xyz_Ca[:, :, 2] - 0.1
    mask = np.all(distance_xyz_Ca < 2.5, axis=1)  # Retain frames where all Ca atoms have z < 1.5 here need adjust according to S2S!!!!!!!
    filtered_xyz_Ca = xyz_Ca[mask]
    filtered_xyz_SC = xyz_SC[mask]
    filtered_PBC = PBC[mask]  # Keep only selected frames' PBC in case of NPT with changing box
    ###### **PBC Correction** ######
    ref_atom_x = filtered_xyz_SC[:, 0, 0]  # Reference x per frame
    ref_atom_y = filtered_xyz_SC[:, 0, 1]  # Reference y per frame
    # Apply periodic boundary corrections per frame
    for atom in range(filtered_xyz_SC.shape[1]):
        # Correct x-coordinates
        mask_x_low = filtered_xyz_SC[:, atom, 0] + filtered_PBC[:, 0] / 2 < ref_atom_x
        mask_x_high = filtered_xyz_SC[:, atom, 0] - filtered_PBC[:, 0] / 2 > ref_atom_x
        filtered_xyz_SC[mask_x_low, atom, 0] += filtered_PBC[mask_x_low, 0]
        filtered_xyz_SC[mask_x_high, atom, 0] -= filtered_PBC[mask_x_high, 0]
        # Correct y-coordinates
        mask_y_low = filtered_xyz_SC[:, atom, 1] + filtered_PBC[:, 1] / 2 < ref_atom_y
        mask_y_high = filtered_xyz_SC[:, atom, 1] - filtered_PBC[:, 1] / 2 > ref_atom_y
        filtered_xyz_SC[mask_y_low, atom, 1] += filtered_PBC[mask_y_low, 1]
        filtered_xyz_SC[mask_y_high, atom, 1] -= filtered_PBC[mask_y_high, 1]
    ###### **Compute CA Distance** ######
    CA_dis = filtered_xyz_Ca[:, :, 2] - 0.1
    return CA_dis, filtered_xyz_SC

In [6]:
class LocalGlobal:
    def __init__(self, s2s_SC_indices):
        N_Ca_C_idx = [s2s_SC_indices[0],s2s_SC_indices[2],s2s_SC_indices[-2]]
        str2str_N_Ca_C = traj_flipped.xyz[:, N_Ca_C_idx, :]
        origin, axes = self.build_local_frame_all(str2str_N_Ca_C[:,0], str2str_N_Ca_C[:,1], str2str_N_Ca_C[:,2])
        self.str2str_local = self.transform_to_local(str2str_N_Ca_C)
        self.origin = origin
        self.axes = axes
        
    def transform_to_local(self, coords, batch_size=100000):
        num_residues = coords.shape[0]
        transformed_coords = np.zeros_like(coords)
        for i in range(0, num_residues, batch_size):
            batch_coords = coords[i:i+batch_size]  # Shape: (batch_size, 3, 3)
            n = batch_coords[:, 0]  # Shape: (batch_size, 3)
            ca = batch_coords[:, 1]  # Shape: (batch_size, 3)
            c = batch_coords[:, 2]  # Shape: (batch_size, 3)
            # Compute local frame for each residue in batch
            origin, axes = self.build_local_frame_all(n, ca, c)  # origin: (batch_size, 3), axes: (batch_size, 3, 3)
            # Reshape origin for broadcasting
            origin = origin[:, np.newaxis, :]  # Shape: (batch_size, 1, 3)
            # Apply transformation using batch-wise matrix multiplication
            transformed_coords[i:i+batch_size] = np.einsum('bij,bjk->bik', batch_coords - origin, axes.transpose(0, 2, 1))
        return transformed_coords

    def build_local_frame_all(self, n, ca, c):
        x_axis = n - ca  # Shape: (batch_size, 3)
        x_axis /= np.linalg.norm(x_axis, axis=1, keepdims=True)  # Normalize
        z_axis = np.cross(x_axis, c - ca)  # Shape: (batch_size, 3)
        z_axis /= np.linalg.norm(z_axis, axis=1, keepdims=True)  # Normalize
        y_axis = np.cross(z_axis, x_axis)  # Shape: (batch_size, 3)
        axes = np.stack([x_axis, y_axis, z_axis], axis=1)  # Shape: (batch_size, 3, 3)
        return ca, axes

    def transform_to_local_MD(self, coords, batch_size=100000):
        num_residues = coords.shape[0]
        transformed_coords = np.zeros_like(coords)
        for i in range(0, num_residues, batch_size):
            batch_coords = coords[i:i+batch_size]  # Shape: (batch_size, 3, 3)
            n = batch_coords[:, 0]  # Shape: (batch_size, 3)
            ca = batch_coords[:, 2]  # Shape: (batch_size, 3)
            c = batch_coords[:, -2]  # Shape: (batch_size, 3)
            # Compute local frame for each residue in batch
            origin, axes = self.build_local_frame_all(n, ca, c)  # origin: (batch_size, 3), axes: (batch_size, 3, 3)
            # Reshape origin for broadcasting
            origin = origin[:, np.newaxis, :]  # Shape: (batch_size, 1, 3)
            # Apply transformation using batch-wise matrix multiplication
            transformed_coords[i:i+batch_size] = np.einsum('bij,bjk->bik', batch_coords - origin, axes.transpose(0, 2, 1))
        return transformed_coords  
        
    def transform_back_to_global(self, local_coords, origin, axes):
        if axes.ndim == 2:  # If shape is (3,3), expand to (1,3,3)
            axes = np.expand_dims(axes, axis=0)
        if origin.ndim == 2:  # If shape is (batch_size,3), expand to (batch_size,1,3)
            origin = origin[:, np.newaxis, :]
        # Inverse transformation: Multiply with `axes` (not transposed) and add back origin
        global_coords = np.einsum('bij,bjk->bik', local_coords, axes) + origin
        return global_coords

In [7]:
# Sidechain replacement class
class SidechainReplacement:
        def __init__(self, idx):
            self.Ca_indices = S2S_residue_indices[idx]['Ca_indices']
            self.SC_indices = S2S_residue_indices[idx]['SC_indices']
            self.res_name = S2S_residue_indices[idx]['res_name']
            self.xyz = traj_flipped.xyz[:, self.Ca_indices, :].squeeze(0)
            self.distance = self.xyz[:, 2] - Au_shift
            self.idx = idx
            self.LG = LocalGlobal(self.SC_indices)
            self.origin = self.LG.origin
            self.axes = self.LG.axes

        def local_info(self, MD_SC_xyz):
            str2str_C_local = self.LG.str2str_local[0][:, 2]
            MD_SC_local = self.LG.transform_to_local_MD(MD_SC_xyz)
            MD_C_local = MD_SC_local[:, -2, :]
            c_align = np.linalg.norm(MD_C_local - str2str_C_local, axis=1)
            MD_SC_global = self.LG.transform_back_to_global(MD_SC_local, self.origin, self.axes)
            return c_align, MD_SC_global

        def sidechain(self, best_sc):
            SC_indices = S2S_residue_indices[self.idx]['SC_indices']
            filtered_tra_SC = traj_flipped.atom_slice(SC_indices)
            old_sc = filtered_tra_SC.xyz
            new_sc = np.expand_dims(best_sc, axis=0)  # Shape (1, n_atoms, 3)
            align_sc = old_sc
            align_sc[:, 4:-2, :] = new_sc[:, 4:-2, :]
            xyz_flipped[:, SC_indices, :] = align_sc

In [8]:
results_dir = "./Results" 
results_dir_merged = "./Results/merged" 
cache_dir = "./Cache"

#z_adjust = np.load('z_adjust.npy')

os.makedirs(cache_dir, exist_ok=True)  # Ensure the Cache directory exists
os.makedirs(results_dir_merged, exist_ok=True)
os.makedirs(results_dir, exist_ok=True) # Ensure Results directory exists os.makedirs(cache_dir, exist_ok=True) 
# Ensure Cache directory exists Find all merged and reset .gro files

gro_files = [f for f in os.listdir(cache_dir) if f.endswith("_Gol_merged_reset.gro")]
gro_files = natsorted(gro_files)

Au_shift = 0.1 # Gold layer shift 
Au_Z = 4.0 # Gold surface at 4.0 nm
z_min = 0.10 
z_max_ad = 0.80 
align_weights = [1, 2]

#below_index = np.loadtxt('below_0_indices.txt')
#if below_index.ndim == 0:
#    below_index = np.array([below_index])  # handle case of single entry

for gro_file in natsorted(gro_files):
    index = int(gro_file.split('_')[-4])
    #print(index)

    #if index not in below_index:
    #    continue  # Skip if index is not in the list, this is for below 0

    #if index in below_index:
    #    continue  # Skip if index is in the list, this is for above 0

    gro_file_path = os.path.join(cache_dir, gro_file)
    base_name = gro_file.replace("_Gol_merged_reset.gro", "")

    # Load trajectory
    traj_new = md.load(gro_file_path)

    # Flip coordinates along z-axis
    xyz_flipped = traj_new.xyz.copy()
    xyz_flipped[:, :, 2] *= -1  # Flip z-coordinates
    xyz_flipped[:, :, 2] += (Au_Z + Au_shift)  # Align with gold surface

    traj_flipped = traj_new
    traj_flipped.xyz = xyz_flipped

    # Save flipped structure
    flipped_gro_path = os.path.join(cache_dir, gro_file.replace("_Gol_merged_reset.gro", "_flipped.gro"))
    traj_flipped.save_gro(flipped_gro_path)

    for idx in peptide_sc:
        sr = SidechainReplacement(idx)
        S2S_CA_dis = sr.distance.reshape(1, 3)
        res_name = sr.res_name

        # Load MD data for sidechain alignment
        MD_CA_dis, MD_SC_xyz = load_MD(res_name)

        # Align based on Ca-Ca-Ca distances
        if idx == peptide_sc[0]:
            norm_align = np.linalg.norm(MD_CA_dis[:, 1:] - S2S_CA_dis[:, 1:], axis=1)
        elif idx == peptide_sc[-1]:
            norm_align = np.linalg.norm(MD_CA_dis[:, :-1] - S2S_CA_dis[:, :-1], axis=1)
        else:
            norm_align = np.linalg.norm(MD_CA_dis - S2S_CA_dis, axis=1)

        # Align using N_Ca_C local info
        c_align, MD_SC_global = sr.local_info(MD_SC_xyz)
        align = norm_align * align_weights[0] + c_align * align_weights[1]

        # Z-cutoff filtering
        z_max = S2S_CA_dis[:, 1] + z_max_ad
        MD_SC_global_z = MD_SC_global[:, 4:-2, 2]
        valid_indices = np.where(
            (np.all(MD_SC_global_z > z_min, axis=1)) & (np.all(MD_SC_global_z < z_max, axis=1))
        )[0]

        if valid_indices.size > 0:
            index_best = valid_indices[np.argmin(align[valid_indices])]
            #print(f"Min Z in valid frames: {np.min(MD_SC_global_z[valid_indices])}")
        else:
            index_best = align.argmin()  # Fallback: Choose best overall
            print('cannot find a match')

        best_sc = MD_SC_global[index_best, :, :]
        print(f"Best sidechain index: {index_best}")
        sr.sidechain(best_sc)

    # Save final reconstructed file
    final_gro_path = os.path.join(results_dir, gro_file.replace("_Gol_merged_reset.gro", "_reconstructed.gro"))
    traj_flipped.xyz = xyz_flipped
    traj_flipped.save_gro(final_gro_path)

    # Split into Au and Peptide structures
    Au = traj_flipped.atom_slice(np.arange(0, 3136))
    box_size = np.array([[4.10200, 4.05993, 4.95517]])
    box_angles = np.array([[90.0, 90.0, 90.0]])
    Au.unitcell_angles = box_angles
    Au.unitcell_lengths = box_size
    Pep = traj_flipped.atom_slice(np.arange(3136, 3258))

    au_gro_path = os.path.join(results_dir, "Au.gro")
    pep_gro_path = os.path.join(results_dir, "Pep.gro")

    Au.save_gro(au_gro_path)
    Pep.save_gro(pep_gro_path)

    #print(f"Processed {gro_file} -> {final_gro_path}")

    # Step 1: Generate index file to remove hydrogens
    index_file_path = os.path.join(results_dir, "index.ndx")
    subprocess.run(
        ["gmx", "make_ndx", "-f", pep_gro_path, "-o", index_file_path],
        input="q\n", text=True, check=True
    )

    # Step 2: Remove all hydrogen atoms (Select **group 2**)
    noH_gro_path = os.path.join(results_dir, "noH.gro")
    subprocess.run(
        ["gmx", "trjconv", "-s", pep_gro_path, "-f", pep_gro_path, "-o", noH_gro_path, "-novel", "-n", index_file_path],
        input="2\n", text=True, check=True
    )

    # Step 3: Re-add hydrogens using pdb2gmx with SPC water model
    h_gro_path = os.path.join(results_dir, "h.gro")
    subprocess.run(
        ["gmx", "pdb2gmx", "-f", noH_gro_path, "-o", h_gro_path, "-ff", "oplsaa", "-water", "spc"],
        check=True
    )

    for file in glob.glob(os.path.join(results_dir, "#*#")):
        os.remove(file)

    for file in glob.glob(os.path.join(os.getcwd(), "#*#")):
        os.remove(file)

    # Step 4: Merge Au and modified peptide into final system
    merged_gro_path = os.path.join(results_dir_merged, f"{base_name}_Gol_merged.gro")
    with open(au_gro_path, "r") as f:
        gol_lines = f.readlines()
    with open(h_gro_path, "r") as f:
        klv_lines = f.readlines()

    header = gol_lines[0]    # Extract headers, atom count, and box size
    box_size_line = gol_lines[-1]
    num_atoms_gol = int(gol_lines[1].strip())

    gol_molecules = gol_lines[2:-1]    # Extract molecular data
    klv_molecules = klv_lines[2:-1]
    total_atoms = num_atoms_gol + len(klv_molecules)

    with open(merged_gro_path, "w") as f: # Write final merged structure
        f.write(header)
        f.write(f" {total_atoms}\n")
        f.writelines(gol_molecules)
        f.writelines(klv_molecules)
        f.write(box_size_line)

Best sidechain index: 1489334


/home/yanbin/miniconda3/envs/MD_analysis/lib/python3.13/site-packages/mdtraj/formats/gro.py:364: UserWarning: WARNING: two consecutive residues with same number (LEU, ACE)
  warnings.warn(
/home/yanbin/miniconda3/envs/MD_analysis/lib/python3.13/site-packages/mdtraj/formats/gro.py:364: UserWarning: WARNING: two consecutive residues with same number (NAC, LEU)
  warnings.warn(


Best sidechain index: 315892
Best sidechain index: 1082404


/home/yanbin/miniconda3/envs/MD_analysis/lib/python3.13/site-packages/mdtraj/formats/gro.py:364: UserWarning: WARNING: two consecutive residues with same number (PHE, ACE)
  warnings.warn(
/home/yanbin/miniconda3/envs/MD_analysis/lib/python3.13/site-packages/mdtraj/formats/gro.py:364: UserWarning: WARNING: two consecutive residues with same number (NAC, PHE)
  warnings.warn(


Best sidechain index: 1788589
Best sidechain index: 1710060


...

/home/yanbin/miniconda3/envs/MD_analysis/lib/python3.13/site-packages/mdtraj/formats/gro.py:364: UserWarning: WARNING: two consecutive residues with same number (PHE, ACE)
  warnings.warn(
/home/yanbin/miniconda3/envs/MD_analysis/lib/python3.13/site-packages/mdtraj/formats/gro.py:364: UserWarning: WARNING: two consecutive residues with same number (NAC, PHE)
  warnings.warn(


Best sidechain index: 416939
Best sidechain index: 747123


/home/yanbin/miniconda3/envs/MD_analysis/lib/python3.13/site-packages/mdtraj/formats/gro.py:364: UserWarning: WARNING: two consecutive residues with same number (GLU, ACE)
  warnings.warn(
/home/yanbin/miniconda3/envs/MD_analysis/lib/python3.13/site-packages/mdtraj/formats/gro.py:364: UserWarning: WARNING: two consecutive residues with same number (NAC, GLU)
  warnings.warn(


Best sidechain index: 31041
Processed KLVFFAE_2_AA_100_Gol_merged_reset.gro -> ./Results/KLVFFAE_2_AA_100_reconstructed.gro
Going to read 0 old index file(s)
Analysing residue names:
There are:     7    Protein residues
Analysing Protein...

  0 System              :   122 atoms
  1 Protein             :   122 atoms
  2 Protein-H           :    60 atoms
  3 C-alpha             :     7 atoms
  4 Backbone            :    21 atoms
  5 MainChain           :    28 atoms
  6 MainChain+Cb        :    35 atoms
  7 MainChain+H         :    35 atoms
  8 SideChain           :    87 atoms
  9 SideChain-H         :    32 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr         'splitres' nr   'l': list residues
 't': atom type  '|': or   'keep' nr        'splitat' nr    'h': help
 'r': residue              'res' nr         'chain' char
 "name": group             'case': case sensitive           'q': save and quit
 'ri': residu

              :-) GROMACS - gmx make_ndx, 2022.5-plumed_2.9.0 (-:

Executable:   /home/yanbin/Apps/gromacs-2022.5/build/bin/gmx
Data prefix:  /home/yanbin/Apps/gromacs-2022.5 (source tree)
Working dir:  /home/yanbin/Desktop/Projects/GolP/Full_data_code/4_18/Au_F_2/2_Z
Command line:
  gmx make_ndx -f ./Results/Pep.gro -o ./Results/index.ndx


Reading structure file

Back Off! I just backed up ./Results/index.ndx to ./Results/#index.ndx.1#

GROMACS reminds you: "Read me your scripture and I will twist it" (Red Hot Chili Peppers)

               :-) GROMACS - gmx trjconv, 2022.5-plumed_2.9.0 (-:

Executable:   /home/yanbin/Apps/gromacs-2022.5/build/bin/gmx
Data prefix:  /home/yanbin/Apps/gromacs-2022.5 (source tree)
Working dir:  /home/yanbin/Desktop/Projects/GolP/Full_data_code/4_18/Au_F_2/2_Z
Command line:
  gmx trjconv -s ./Results/Pep.gro -f ./Results/Pep.gro -o ./Results/noH.gro -novel -n ./Results/index.ndx

Will write gro: Coordinate file in Gromos-87 format
Group     0 (         S